In [ ]:
# Colab에 Mecab 설치
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 91, done.
remote: Counting objects: 100% (91/91), done.
remote: Compressing objects: 100% (85/85), done.
remote: Total 91 (delta 43), reused 22 (delta 6), pack-reused 0
Unpacking objects: 100% (91/91), done.
/content/Mecab-ko-for-Google-Colab
Installing konlpy.....
     |████████████████████████████████| 19.4 MB 57.7 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 448 kB 59.4 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2021-08-03 00:51:05--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.141.1, 2406:da00:ff0

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import urllib.request
from collections import Counter
from konlpy.tag import Mecab
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.tokenize import RegexpTokenizer
import numpy as np
import matplotlib.pyplot as plt
import io

df=pd.read_csv('/content/drive/My Drive/your data',  error_bad_lines=False)

df.head(10)

,Unnamed: 0,Place,Address,Name,Review,Date,Score
0,0,전주 한옥 마을,-,diditall,전주가면 무조건 가봐야하는곳한옥하나하나 너무 멋지고 잘 꾸며져 있어요그러나 길거리 ...,2021년 5월 25일,4.0
1,1,전주 한옥 마을,-,hyun,"젊음의 거리, 사진찍는 거리입니다. 저렴하게 빌리는 옷 덕분에 여기 저기 한복 입고...",2021년 5월 6일,4.0
2,2,전주 한옥 마을,-,l*alalala,한복입고 사진찍긴 좋아요거리음식거리라서 먼가 특별하다 싶지 않아요물론 처음가서 보기...,2021년 5월 4일,3.0
3,3,전주 한옥 마을,-,ashurei2000,전주여행을 생각한다면 꼭 한번 가게되는 곳이라고 생각합니다.한옥을 보는 재미도 있지...,2020년 9월 18일,4.0
4,4,전주 한옥 마을,-,blessmj,첫째날은 사람구경둘째날은 먹거리 먹고 꽃구경 셋째날은 세세한 곳 소리관 김치 그...,2020년 5월 28일,5.0
5,5,전주 한옥 마을,-,사장님,전주 한옥마을을 평일에 갔더니 한산하니 좋았어요.길에 쓰레기 하나 없이 너무 깨끗했...,2020년 2월 4일,3.0
6,6,전주 한옥 마을,-,composuretour,명절 긴 이동거리를 고려하여 중간 휴식지로 전주 한옥마을을 선택명절 직전 한옥 마을...,2020년 1월 29일,3.0
7,7,전주 한옥 마을,-,Oh!,한옥마을 거리가 온통 가게들로 바뀌었네요. 사람이 북적거리는 건 좋은데 고즈넉한 맛...,2020년 1월 14일,3.0
8,8,전주 한옥 마을,-,panclass,"오랜만에 가봤는 데, 깨끗하게 관리 되고 있는 듯 보였고, 이쁜 한복체험하는 곳도 ...",2020년 1월 2일,5.0
9,9,전주 한옥 마을,-,ahirity,전주에 내려간 여행객들이 모두 모이는 곳이다. 먹자거리가 형성되어 있어서 길을 걷다...,2019년 12월 9일,4.0


In [ ]:
cdf=df[['Review']]
print(type(cdf))

<class 'pandas.core.frame.DataFrame'>


In [ ]:
drop_index = cdf[cdf['Review'] == '-'].index
cdf=cdf.drop(drop_index)
cdf.head()

,Review
0,전주가면 무조건 가봐야하는곳한옥하나하나 너무 멋지고 잘 꾸며져 있어요그러나 길거리 ...
1,"젊음의 거리, 사진찍는 거리입니다. 저렴하게 빌리는 옷 덕분에 여기 저기 한복 입고..."
2,한복입고 사진찍긴 좋아요거리음식거리라서 먼가 특별하다 싶지 않아요물론 처음가서 보기...
3,전주여행을 생각한다면 꼭 한번 가게되는 곳이라고 생각합니다.한옥을 보는 재미도 있지...
4,첫째날은 사람구경둘째날은 먹거리 먹고 꽃구경 셋째날은 세세한 곳 소리관 김치 그...


In [ ]:
cdf_l=list(np.array(cdf.Review.tolist()))

In [ ]:
print(cdf_l[0])

전주가면 무조건 가봐야하는곳한옥하나하나 너무 멋지고 잘 꾸며져 있어요그러나 길거리 음식들이 너무 난잡하게 되어있어요간단하게 둘러보시고 사진찍기 좋아요


In [ ]:
import re
clean_text = []

for n in range(len(cdf_l)):
    
    try:
      line = cdf_l[n]
      l = re.compile('[^ ㄱ-ㅣ가-힣]+') # 한글을 뺀 나머지 l에 저장
      text = l.sub("",line) # l을 line에서 지움
      clean_text.append(text)
    except : pass
  
print(clean_text)

['전주가면 무조건 가봐야하는곳한옥하나하나 너무 멋지고 잘 꾸며져 있어요그러나 길거리 음식들이 너무 난잡하게 되어있어요간단하게 둘러보시고 사진찍기 좋아요', '젊음의 거리 사진찍는 거리입니다 저렴하게 빌리는 옷 덕분에 여기 저기 한복 입고 다니며 사진 찍는 사람들이 많읍니다 옷도 예쁘고 머리 장식까지 예쁘게 잘 해주는데 한 번쯤 해볼만 합니다 나이 드신 분들도 여고 동창들 끼리 모녀들도 와서 하느데 보기 좋더라구요  길거리 음식도 저렴하고 맛있읍니다', '한복입고 사진찍긴 좋아요거리음식거리라서 먼가 특별하다 싶지 않아요물론 처음가서 보기에는 좋고 골목골목돌아다니면    길이나 돌담이나 예쁘긴 해요하지만 거리음식거리같아요 어느순간 한옥마을은 사라지고 음식점만 남았엉ㄷ', '전주여행을 생각한다면 꼭 한번 가게되는 곳이라고 생각합니다한옥을 보는 재미도 있지만길거리 여러가지 음식들도 여행의 즐거움을 더해줍니다특히나 육전과 문어꼬치가 맛있었어요그리고 한국 전통의 얼레빗을 파는 매장을 발견했는데 무형문화재 이상근목소장님이하나 하나 만드시는 것으로 너무 아름답고 좋았습니다전주 마을을 가신다면 방문해보시고 선물용으로 장만하셔도 좋을거 같아요저도 가격이 좀 비쌌지만 마음에 드는 얼레빗을 장만했지요', '첫째날은 사람구경둘째날은 먹거리 먹고 꽃구경 셋째날은 세세한 곳 소리관 김치   그릇구경등  볼거리가 소소하게 많아 좋았어요', '전주 한옥마을을 평일에 갔더니 한산하니 좋았어요길에 쓰레기 하나 없이 너무 깨끗했고 주차장도 잘 되어 있어서 구경하기에 좋았답니다요즘엔 한복뿐만 아니라 경성의복 대여점도 있어 경성의복 대여해서 입고 다니는 분들도 많아서 잼있었어요', '명절 긴 이동거리를 고려하여 중간 휴식지로 전주 한옥마을을 선택명절 직전 한옥 마을 그리고 한옥 펜션을 구경하니 더욱 명절 스럽고나름 의미있는 기분 전환의 시간을 보낼 수 있음', '한옥마을 거리가 온통 가게들로 바뀌었네요 사람이 북적거리는 건 좋은데 고즈넉한 맛은 사라져서 아쉽기만 하네요 정체불명의 한복들도 많이 보

In [ ]:
!pip install git+https://github.com/ssut/py-hanspell.git

  Cloning https://github.com/ssut/py-hanspell.git to /tmp/pip-req-build-1n0x6v6q
  Running command git clone -q https://github.com/ssut/py-hanspell.git /tmp/pip-req-build-1n0x6v6q
  Created wheel for py-hanspell: filename=py_hanspell-1.1-py3-none-any.whl size=4870 sha256=ee79e623080aed0f79307be6caee2373839d0676ae558a0a360d04449fb023fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-0setejn2/wheels/ab/f5/7b/d4124bb329c905301baed80e2ae45aa14e824f62ebc3ec2cc4
Successfully built py-hanspell


In [ ]:
from hanspell import spell_checker

sent = "맞춤법 틀리면 외 않되? 쓰고싶은대로쓰면돼지 "
spelled_sent = spell_checker.check(sent)
hanspell_sent = spelled_sent.checked


print(hanspell_sent)

맞춤법 틀리면 왜 안돼? 쓰고 싶은 대로 쓰면 되지


In [ ]:
hansp=[]
for i in clean_text:
  print(i)
  sent = i
  spelled_sent = spell_checker.check(sent)
  hanspell_sent = spelled_sent.checked
  hansp.append(hanspell_sent)

print(hansp)

전주가면 무조건 가봐야하는곳한옥하나하나 너무 멋지고 잘 꾸며져 있어요그러나 길거리 음식들이 너무 난잡하게 되어있어요간단하게 둘러보시고 사진찍기 좋아요
젊음의 거리 사진찍는 거리입니다 저렴하게 빌리는 옷 덕분에 여기 저기 한복 입고 다니며 사진 찍는 사람들이 많읍니다 옷도 예쁘고 머리 장식까지 예쁘게 잘 해주는데 한 번쯤 해볼만 합니다 나이 드신 분들도 여고 동창들 끼리 모녀들도 와서 하느데 보기 좋더라구요  길거리 음식도 저렴하고 맛있읍니다
한복입고 사진찍긴 좋아요거리음식거리라서 먼가 특별하다 싶지 않아요물론 처음가서 보기에는 좋고 골목골목돌아다니면    길이나 돌담이나 예쁘긴 해요하지만 거리음식거리같아요 어느순간 한옥마을은 사라지고 음식점만 남았엉ㄷ
전주여행을 생각한다면 꼭 한번 가게되는 곳이라고 생각합니다한옥을 보는 재미도 있지만길거리 여러가지 음식들도 여행의 즐거움을 더해줍니다특히나 육전과 문어꼬치가 맛있었어요그리고 한국 전통의 얼레빗을 파는 매장을 발견했는데 무형문화재 이상근목소장님이하나 하나 만드시는 것으로 너무 아름답고 좋았습니다전주 마을을 가신다면 방문해보시고 선물용으로 장만하셔도 좋을거 같아요저도 가격이 좀 비쌌지만 마음에 드는 얼레빗을 장만했지요
첫째날은 사람구경둘째날은 먹거리 먹고 꽃구경 셋째날은 세세한 곳 소리관 김치   그릇구경등  볼거리가 소소하게 많아 좋았어요
전주 한옥마을을 평일에 갔더니 한산하니 좋았어요길에 쓰레기 하나 없이 너무 깨끗했고 주차장도 잘 되어 있어서 구경하기에 좋았답니다요즘엔 한복뿐만 아니라 경성의복 대여점도 있어 경성의복 대여해서 입고 다니는 분들도 많아서 잼있었어요
명절 긴 이동거리를 고려하여 중간 휴식지로 전주 한옥마을을 선택명절 직전 한옥 마을 그리고 한옥 펜션을 구경하니 더욱 명절 스럽고나름 의미있는 기분 전환의 시간을 보낼 수 있음
한옥마을 거리가 온통 가게들로 바뀌었네요 사람이 북적거리는 건 좋은데 고즈넉한 맛은 사라져서 아쉽기만 하네요 정체불명의 한복들도 많이 보이구요
오랜만에 가봤는 데 깨끗하게 관리 

In [ ]:
!pip install soynlp

     |████████████████████████████████| 416 kB 8.3 MB/s 


In [ ]:
import urllib.request
from soynlp import DoublespaceLineCorpus
from soynlp.word import WordExtractor
urllib.request.urlretrieve("https://raw.githubusercontent.com/lovit/soynlp/master/tutorials/2016-10-20.txt", filename="2016-10-20.txt")

('2016-10-20.txt', <http.client.HTTPMessage at 0x7ff0625af390>)

In [ ]:
corpus = DoublespaceLineCorpus("2016-10-20.txt")
len(corpus)

30091

In [ ]:
i = 0
for document in corpus:
  if len(document) > 0:
    print(document)
    i = i+1
  if i == 10:
    break

19  1990  52 1 22
오패산터널 총격전 용의자 검거 서울 연합뉴스 경찰 관계자들이 19일 오후 서울 강북구 오패산 터널 인근에서 사제 총기를 발사해 경찰을 살해한 용의자 성모씨를 검거하고 있다 성씨는 검거 당시 서바이벌 게임에서 쓰는 방탄조끼에 헬멧까지 착용한 상태였다 독자제공 영상 캡처 연합뉴스  서울 연합뉴스 김은경 기자 사제 총기로 경찰을 살해한 범인 성모 46 씨는 주도면밀했다  경찰에 따르면 성씨는 19일 오후 강북경찰서 인근 부동산 업소 밖에서 부동산업자 이모 67 씨가 나오기를 기다렸다 이씨와는 평소에도 말다툼을 자주 한 것으로 알려졌다  이씨가 나와 걷기 시작하자 성씨는 따라가면서 미리 준비해온 사제 총기를 이씨에게 발사했다 총알이 빗나가면서 이씨는 도망갔다 그 빗나간 총알은 지나가던 행인 71 씨의 배를 스쳤다  성씨는 강북서 인근 치킨집까지 이씨 뒤를 쫓으며 실랑이하다 쓰러뜨린 후 총기와 함께 가져온 망치로 이씨 머리를 때렸다  이 과정에서 오후 6시 20분께 강북구 번동 길 위에서 사람들이 싸우고 있다 총소리가 났다 는 등의 신고가 여러건 들어왔다  5분 후에 성씨의 전자발찌가 훼손됐다는 신고가 보호관찰소 시스템을 통해 들어왔다 성범죄자로 전자발찌를 차고 있던 성씨는 부엌칼로 직접 자신의 발찌를 끊었다  용의자 소지 사제총기 2정 서울 연합뉴스 임헌정 기자 서울 시내에서 폭행 용의자가 현장 조사를 벌이던 경찰관에게 사제총기를 발사해 경찰관이 숨졌다 19일 오후 6시28분 강북구 번동에서 둔기로 맞았다 는 폭행 피해 신고가 접수돼 현장에서 조사하던 강북경찰서 번동파출소 소속 김모 54 경위가 폭행 용의자 성모 45 씨가 쏜 사제총기에 맞고 쓰러진 뒤 병원에 옮겨졌으나 숨졌다 사진은 용의자가 소지한 사제총기  신고를 받고 번동파출소에서 김창호 54 경위 등 경찰들이 오후 6시 29분께 현장으로 출동했다 성씨는 그사이 부동산 앞에 놓아뒀던 가방을 챙겨 오패산 쪽으로 도망간 후였다  김 경위는 오패산 터널 입구 오른쪽의 급경사에서 성씨에

In [ ]:
word_extractor = WordExtractor()
word_extractor.train(corpus)
word_score_table = word_extractor.extract()

training was done. used memory 1.446 Gb
all cohesion probabilities was computed. # words = 223348
all branching entropies was computed # words = 361598
all accessor variety was computed # words = 361598


In [ ]:
from soynlp.tokenizer import MaxScoreTokenizer
scores = {word:score.cohesion_forward for word, score in word_score_table.items()}
maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
maxscore_tokenizer.tokenize("국제사회와우리의노력들로범죄를척결하자")

['국제사회', '와', '우리', '의', '노력', '들로', '범죄', '를', '척결', '하자']

In [ ]:
Soynlp=[]
for i in hansp:
  maxscore_tokenizer = MaxScoreTokenizer(scores=scores)
  m = maxscore_tokenizer.tokenize(i)
  Soynlp.append(m)

print(Soynlp)

[['전주', '가면', '무조건', '가', '봐야', '하는', '곳', '한옥', '하나하나', '너무', '멋지', '고', '잘', '꾸며져', '있어요', '그러나', '길거리', '음식', '들이', '너무', '난잡하게', '되어', '있어요', '간단하게', '둘러', '보시고', '사진', '찍기', '좋아', '요'], ['젊음의', '거리', '사진', '찍는', '거리', '입니다', '저렴하게', '빌리는', '옷', '덕분에', '여기저기', '한복', '입고', '다니며', '사진', '찍는', '사람들이', '많습니다', '옷도', '예쁘고', '머리', '장식', '까지', '예쁘게', '잘', '해주는', '데', '한', '번쯤', '해볼', '만', '합니다', '나이', '드신', '분들도', '여고', '동창들끼리', '모녀들도', '와서', '하는', '데', '보기', '좋더', '라고', '요', '길거리', '음식', '도', '저렴', '하고', '맛', '있습니다'], ['한복', '입고', '사진', '찍긴', '좋아', '요', '거리', '음식', '거리', '라서', '먼가', '특별', '하다', '싶지', '않아', '요', '물론', '처음', '가서', '보기에는', '좋고', '골목골목', '돌아', '다니면', '길이', '나', '돌담이나', '예쁘긴', '해요', '하지만', '거리', '음식', '거리', '같아요', '어느', '순간', '한옥마을', '은', '사라지고', '음식', '점만', '남았어ᄃ'], ['전주', '여행을', '생각한다', '면', '꼭', '한번', '가게', '되는', '곳이', '라고', '생각', '합니다', '한옥을', '보는', '재미도', '있지만', '길거리', '여러', '가지', '음식', '들도', '여행', '의', '즐거움을', '더해', '줍니다', '특히', '나', '육전과', '문어', '꼬치가', '맛있', '었어요', '그리